In [1]:
import os, time, datetime, sys
import numpy as np
import pandas as pd
from collections import Counter
from functools import reduce
from dateutil.relativedelta import relativedelta
from sklearn.metrics import roc_curve, auc, f1_score, mean_squared_error
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MultiLabelBinarizer, OneHotEncoder
import matplotlib.pyplot as plt
import matplotlib
import random
from IPython.display import display

In [2]:
# 读取数据
userinfo_train = pd.read_csv('./userinfo.csv')
bank_train = pd.read_csv('./bank.csv')
bill_train = pd.read_csv('./bill.csv')
overdue_train = pd.read_csv('./overdue.csv')

df_dict = {"用户信息": userinfo_train,
"银行流水": bank_train,
"账单": bill_train,
"违约记录": overdue_train}

In [3]:
for name in df_dict:
    print(name+":", len(df_dict[name]), "行")
    display(df_dict[name].head())

用户信息: 10000 行


,new_user_id,性别,职业,教育程度,婚姻状态,户口类型
0,7259fe272934e0dc348092c37544a445,女,公职,本科及以上,未婚,集体户口
1,0c87f6b1f7e0350a35841ba34df9d26a,男,公职,本科及以上,离异,集体户口
2,74c53a62f15cc039864aefc41170f81b,男,公职,大专,离异,家庭户口
3,211f56f356fa6ecc6334bb73e52c7b16,男,企业,高中,离异,农业户口
4,07b0d9ac7924424894141b2fecfa0ee0,男,公职,本科及以上,已婚,集体户口


银行流水: 1068424 行


,new_user_id,交易时间戳,交易类型,交易金额,工资收入标记,交易时间,交易年,交易月,交易日,交易时,交易分,交易秒
0,65b01fba105ee82613babff7c88929c5,58848407247,1,43.757548,0,2156-06-25 21:58:44,2156,6,25,21,58,44
1,65b01fba105ee82613babff7c88929c5,58849365297,1,42.143743,0,2156-06-27 00:35:29,2156,6,27,0,35,29
2,65b01fba105ee82613babff7c88929c5,58850118237,1,40.189051,0,2156-06-27 21:30:23,2156,6,27,21,30,23
3,65b01fba105ee82613babff7c88929c5,58850118447,1,40.189051,0,2156-06-27 21:30:44,2156,6,27,21,30,44
4,65b01fba105ee82613babff7c88929c5,58850244487,1,40.189051,0,2156-06-28 01:00:48,2156,6,28,1,0,48


账单: 429441 行


,new_user_id,账单时间戳,银行id,上期账单金额,上期还款金额,信用卡额度,本期账单余额,本期账单最低还款额,消费笔数,本期账单金额,...,可用金额,预借现金额度,还款状态,账单时间,账单年,账单月,账单日,账单时,账单分,账单秒
0,eddcaa8984f8db5199ec28323efb18a4,59067443637,6,50.622246,50.657612,52.596975,50.897707,49.846899,1,26.457513,...,26.457513,51.933872,0,2157-03-06 10:19:23,2157,3,6,10,19,23
1,eddcaa8984f8db5199ec28323efb18a4,59067444017,6,50.897707,50.901821,52.596975,51.101179,49.911428,1,26.457513,...,26.457513,51.933872,0,2157-03-06 10:20:01,2157,3,6,10,20,1
2,eddcaa8984f8db5199ec28323efb18a4,59067444277,6,51.101179,51.137354,52.596975,51.283715,49.977605,1,26.457513,...,26.457513,51.933872,0,2157-03-06 10:20:27,2157,3,6,10,20,27
3,eddcaa8984f8db5199ec28323efb18a4,59067445157,6,51.283715,51.283799,52.915867,52.252502,50.474921,1,26.457513,...,26.457513,52.256811,0,2157-03-06 10:21:55,2157,3,6,10,21,55
4,eddcaa8984f8db5199ec28323efb18a4,59067445627,6,52.252502,52.256812,52.915867,52.304048,50.508400,1,26.457513,...,26.457513,52.256811,0,2157-03-06 10:22:42,2157,3,6,10,22,42


违约记录: 10000 行


,new_user_id,标签,分数
0,eddcaa8984f8db5199ec28323efb18a4,0,80.619639
1,65b01fba105ee82613babff7c88929c5,1,53.052357
2,968083a2d1a7e6b5b85a66b0bc412830,1,46.796518
3,18aeed58fd39a58cc1430a3a417ed7a9,1,82.354501
4,bae4896cea4a34616ae3babca11439ae,1,83.758928


In [4]:
for name in df_dict:
    print(name+':', np.any(df_dict[name].isnull()))

用户信息: True
银行流水: False
账单: False
违约记录: False


In [6]:
if np.any(userinfo_train.isnull()):
    print("存在空值")
else:
    print("不存在空值")
# 各列有多少空值
print(userinfo_train.isnull().sum(axis=0))
# 顶多有 null_number 行是空值
null_number = sum(userinfo_train.isnull().sum(axis=0))
# 把所有空值的行显示出来
print(userinfo_train[userinfo_train.isnull().T.any()].head(null_number))

存在空值
new_user_id    0
性别             0
职业             2
教育程度           0
婚姻状态           1
户口类型           0
dtype: int64
                           new_user_id 性别   职业   教育程度 婚姻状态  户口类型
31    3be79a55ccf09062cdff1b9c684179f9  男  NaN  本科及以上   离异  农业户口
135   b074add6f740af33df4230f28d37a333  男  NaN     大专   离异  家庭户口
5345  9f2e0eb314d821f8d075fbc9e64ce4f5  男   公职     高中  NaN  农业户口


In [8]:
for name in df_dict:
    df = df_dict[name]
    if np.any(df.isnull()):
        print(name+"存在空值")
        print(df.isnull().sum(axis=0))
        display(df[df.isnull().any(axis=1)])
        print("存在", len(df[df.isnull().any(axis=1)]), "行空值")

        df_dict[name] = df[~df.isnull().any(axis=1)]
        print(name, "已更新")

用户信息存在空值
new_user_id    0
性别             0
职业             2
教育程度           0
婚姻状态           1
户口类型           0
dtype: int64


,new_user_id,性别,职业,教育程度,婚姻状态,户口类型
31,3be79a55ccf09062cdff1b9c684179f9,男,NaN,本科及以上,离异,农业户口
135,b074add6f740af33df4230f28d37a333,男,NaN,大专,离异,家庭户口
5345,9f2e0eb314d821f8d075fbc9e64ce4f5,男,公职,高中,NaN,农业户口


存在 3 行空值
                           new_user_id 性别    职业   教育程度 婚姻状态  户口类型
0     7259fe272934e0dc348092c37544a445  女    公职  本科及以上   未婚  集体户口
1     0c87f6b1f7e0350a35841ba34df9d26a  男    公职  本科及以上   离异  集体户口
2     74c53a62f15cc039864aefc41170f81b  男    公职     大专   离异  家庭户口
3     211f56f356fa6ecc6334bb73e52c7b16  男    企业     高中   离异  农业户口
4     07b0d9ac7924424894141b2fecfa0ee0  男    公职  本科及以上   已婚  集体户口
...                                ... ..   ...    ...  ...   ...
9995  6b8887024b4258aab9c7b655fd2a718f  男    公职  本科及以上   离异  农业户口
9996  158dd40361cd883d37a30fc448cfa7da  男    公职     大专   未婚  集体户口
9997  12fd60bba16cb55cdf3ca81d7d3326b5  女    公职  本科及以上   离异  其他户口
9998  180232fd2d1db5512be0fe554e58405c  男    企业     大专   未婚  家庭户口
9999  cd1b034b97878a461a471337c099e8d3  男  事业单位  本科及以上   未婚  家庭户口

[9997 rows x 6 columns] 已更新


In [ ]:
def toDict(col):
    """
    这个函数的作用：生成替换字典，加入 col 列有两种值 A B
    则生成：{A: 0, B: 1}  即把 A 替换为 0 ， B 替换为 1
    """
    res = {}
    cnt = 0
    for e in userinfo_train[col].unique():
        res[e] = cnt
        cnt += 1
    return res
# 生成列的替换字典
dict1 = toDict("性别")
dict2 = toDict("职业")
dict3 = toDict("教育程度")
dict4 = toDict("婚姻状态")
dict5 = toDict("户口类型")
# 替换后结果
userinfo_train = userinfo_train.replace({"性别": dict1, "职业": dict2, "教育程度": dict3, "婚姻状态": dict4, "户口类型": dict5})